In [ ]:
from fastplotlib.widgets import ImageWidget
import numpy as np
import tifffile

# Single image sequence

In [ ]:
# using Sue demo movie
path = "/home/kushal/caiman_data/example_movies/Sue_2x_3000_40_-46.tif"
a = tifffile.imread(path)

In [ ]:
iw = ImageWidget(
    data=a, 
    vmin_vmax_sliders=True,
    cmap="gnuplot2"
)

In [ ]:
iw.show()

# Try scrolling along the "t" - time dimension and changing the min-max!

### Play with setting different window functions

These can also be given as kwargs to `ImageWidget` during instantiation

In [ ]:
# must be in the form of {dim: (func, window_size)}
iw.window_funcs = {"t": (np.mean, 5)}

In [ ]:
# change the winow size
iw.window_funcs["t"].window_size = 23

In [ ]:
# change the function
iw.window_funcs["t"].func = np.max

In [ ]:
# or set it again
iw.window_funcs = {"t": (np.min, 11)}

### `ImageWidget` embeds a `Plot` or `GridPlot` that is accessible using `ImageWidget.plot`

In [ ]:
iw.plot

# graphics etc. just like before

In [ ]:
iw.plot.graphics

In [ ]:
iw.plot.graphics[0].cmap="gray"

# Close the canvas

In [ ]:
iw.plot.canvas.close()

# Gridplot of txy data

In [ ]:
dims = (100, 512, 512)
data = [np.random.rand(*dims) for i in range(4)]

In [ ]:
iw = ImageWidget(
    data=data, 
    vmin_vmax_sliders=True,
    names=["zero", "one", "two", "three"],
    window_funcs={"t": (np.mean, 5)},  # you can pass window_funcs as a kwarg
    cmap="gnuplot2", 
)

### pan-zoom controllers are all synced in a `ImageWidget`

In [ ]:
iw.show()

### This `ImageWidget` contains a `GridPlot`, so you can index the subplots using the names given to `ImageWidget`

In [ ]:
iw.plot["two"]

### change window functions just like before

In [ ]:
iw.window_funcs["t"].func = np.max

In [ ]:
# make sure to close plot before moving to the next one
iw.plot.canvas.close()

## Gridplot of volumetric data

In [ ]:
dims = (256, 256, 5, 100)
data = [np.random.rand(*dims) for i in range(4)]

iw = ImageWidget(
    data=data, 
    slider_dims=["t", "z"], 
    dims_order="xyzt", # example of how you can set this for non-standard orders
    vmin_vmax_sliders=True,
    names=["zero", "one", "two", "three"],
    # window_funcs={"t": (np.mean, 5)}, # window functions can be slow when indexing multiple dims
    cmap="gnuplot2", 
)

In [ ]:
iw.show()

### window functions, can be slow when you have "t" and "z"

In [ ]:
iw.window_funcs = {"t": (np.mean, 11)}

In [ ]:
# make sure to close plot
iw.plot.canvas.close()

# Volumetric zebrafish data, view each plane using a `GridPlot`

#### `tifffile.memmap` is a nice way to lazy-load large imaging files. This file is kinda small (10G), but we can load it instantly using `tifffile.memmap`, it would take ~@0 seconds to load into RAM on this laptop with a SATA SSD.

In [ ]:
zfish = tifffile.memmap("/home/kushal/Downloads/zfish_vol.tiff")

In [ ]:
zfish.shape

In [ ]:
iw = ImageWidget(zfish, vmin_vmax_sliders=True, cmap="gnuplot2")
iw.show()

In [ ]:
iw.window_funcs = {"t": (np.mean, 5), "z": None}

In [ ]:
iw.window_funcs = {"t": (np.mean, 5), "z": (np.mean, 10)}

In [ ]:
zfish.shape

In [ ]:
zfish_split = [zfish[:, i, :, :] for i in range(zfish.shape[1])]

In [ ]:
iw = ImageWidget(zfish_split, window_funcs={"t": (np.mean, 5)}, cmap="gnuplot2")
iw.show()

In [ ]:
from mesmerize_core.arrays import LazyTiff

In [ ]:
ll -h /home/kushal/Downloads/20220513_rec1.tif

In [ ]:
movie = LazyTiff("/home/kushal/Downloads/20220513_rec1.tif")

In [ ]:
iw = ImageWidget(movie, vmin_vmax_sliders=True, cmap="gnuplot2")
iw.show()

In [ ]:
# workaround for now until there's a permanent fix with LazyTiff
# use frame_apply and set the window averaging manually.
window_size = 23
flank = int((window_size - 1) / 2)

iw.frame_apply = {0: lambda f: np.mean(movie[iw.current_index["t"] - flank:iw.current_index["t"] + flank], axis=0)}

# Kill the kernel before moving on to the next demo!